In [ ]:
import requests

## Simple API Call with Requests Library

It may be good to look at the reference documentation for the [requests library](https://2.python-requests.org/en/master/user/quickstart/).

First, let's have a look at the [GitHub API](https://developer.github.com/v3/).

In [ ]:
r = requests.get('https://api.github.com/users/nmattei', timeout=10)
r.status_code

In [ ]:
r.headers['content-type']

In [ ]:
r.url

In [ ]:
r.content

In [ ]:
r.json()

## Looking at HTTP Requests

We'll try to get some data from Google.  Note that this is kind of against the TOS and we **should not do it this way in general -- Google has very [specific rules on their site](https://developers.google.com/custom-search/v1/).**

In [ ]:
params = {'q':'Tulane University'}
r = requests.get('http://www.google.com/search', params = params, timeout=10)
r.status_code

In [ ]:
r.url

In [ ]:
r.headers['content-type']

In [ ]:
r.text

## More Complicated with Parameters

We'll look for some information from the [Apple ITunes API](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/).

In [ ]:
params = {'term' : "the+meters"}
r = requests.get('https://itunes.apple.com/search', params=params, timeout=10)
r.status_code

In [ ]:
r.url

In [ ]:
r.json()

In [ ]:
r.url

We can do lots of parameters in the payload like [this](https://2.python-requests.org/en/master/user/quickstart/).

In [ ]:
params = {'term' : "the+meters", 'entity' : 'album'}
r = requests.get('https://itunes.apple.com/search', params=params, timeout=10)
r.status_code


In [ ]:
r.url

In [ ]:
x = r.json()

In [ ]:
x['results'][0]['wrapperType']

## Converting the returned JSON to an object!

In [ ]:
import json

In [ ]:
data = json.loads(r.content)

In [ ]:
data.keys()

In [ ]:
data['results'][1]

## Using Beautiful Soup to Parse a Webpage.

The [beautifulsoup4 documentation](https://www.crummy.com/software/BeautifulSoup/).

In [ ]:
# Grab the course webpage.
import requests
from bs4 import BeautifulSoup

r = requests.get('https://tulaneintrodatascience.github.io/')

root = BeautifulSoup( r.content )

In [ ]:
r.content

In [ ]:
root.find("table")

In [ ]:
root.find("table").findAll("a")

## Trying out some Regular Expressions.

In [ ]:
import re
# Find the index in the raw HTML where we first mention CMPS3660

# Note we use the r to make sure special flags get used correctly.

match = re.search(r'CMPS 3660', r.text)
print(match.start())

In [ ]:
r.text[410:450]

In [ ]:
# Does the start match?
match = re.match(r'CMPS 3660', r.text)
print(match)

In [ ]:
# Iterate over all occurances and print a few characters.
for match in re.finditer(r'CMPS 3660', r.text):
    print(r.text[match.start()-30:match.start()+20])


In [ ]:
# Find them all.
match = re.findall(r'CMPS 3660', r.text)
print(match)

In [ ]:
# More complicated RegExes - Groups
regex = r'\s*([Uu]niversity)\s([Oo]f)\s(\w{3,})'
text = ''' The university of kentucky is the best
            basketball team and an ok university.
            The University Of Kentucky can be put in 
            some weird capitalization'''
m = re.search( regex, text)
print(m.groups())

In [ ]:
# Find all
print(re.findall(regex, text))

In [ ]:
# Named Groups.
regex = r'\s*([Uu]niversity)\s([Oo]f)\s(?P<school>\w{3,})'
text = ''' The university of kentucky is the best
            basketball team and an ok university.
            The University Of Kentucky can be put in 
            some weird capitalization'''
m = re.search( regex, text)
print(m.groupdict())


In [ ]:
# Find all named groups

# Named Groups.
regex = r'\s*([Uu]niversity)\s([Oo]f)\s(?P<school>\w{3,})'
text = ''' The university of kentucky is the best
            basketball team and an ok university.
            The University Of Kentucky can be put in 
            some weird capitalization.  And Kentucky is much better than
            the University of Mississippi.'''
for m in re.finditer(regex, text):
    print(m.groupdict())


In [ ]:
'abcabcabc'.replace('a', 'X')

In [ ]:
text = 'I love Introduction to Data Science'
re.sub(r'Data Science', r'Schmada Schmience', text) 

In [ ]:
re.sub(r'(\w+)\s([Ss]cience)', r'\2 \1hmience', text) 


## Downloadning All the PDFs from the course website.

Using beautiful soup and some regular expressions.

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.parse import urljoin
import os

In [ ]:
# HTTP GET request sent to the URL url
url = "https://tulaneintrodatascience.github.io/"
r = requests.get( url )

# Use BeautifulSoup to parse the GET response
root = BeautifulSoup( r.content )
lnks = root.find("table").find("tbody").findAll("a")
lnks

In [ ]:
# Cycle through the href for each anchor, checking
# to see if it's a PDF/PPTX link or not
pdfs = []
for lnk in lnks:
    href = lnk['href']
    
    # If it's a PDF/PPTX link, queue a download   
    if href.lower().endswith(('.pdf', '.pptx')):
        pdfs.append(href)
print(pdfs)

In [ ]:
# Download all the files to whatever you're running notebook from.

# Be careful for href!

for href in pdfs:
    urld = urljoin(url, href)
    print(urld)
    rd = requests.get(urld, stream=True)
    
    # Write the downloaded PDF to a file
    # Note because the href is a path we have to just get the filename!
    outfile = os.path.join("./", href.split("/")[2])
    print("Writing: ",outfile)
    with open(outfile, 'wb') as f:
        f.write(rd.content)
